In [1]:
from scipy.optimize import root
import numpy as np
import pickle

In [2]:
with open('mf1.pkl', 'rb') as f:
    ss_params,params,ss_shock = pickle.load(f)


We need a few of the model parameters to solve for the steady state of the model. For simplicity, we run this to define them all. 

In [3]:
%run par_list.m


We can find the steady state analyticall but do it numerically. We are solving only for exp(c), exp(y), i, and exp(k). We do not exonentiate i as we must allow for it to be negative.  This is give the system of four equations and four unknowns to solve. 

In [4]:
def equations(vars):
	y,c,i,k = vars
	#prodction function
	eq1=np.exp(y)-np.exp(a_0)*np.exp(k)**alpha
	#resource constraint	
	eq2=np.exp(y)-np.exp(c)-i
	#capital accumulation 
	eq3=np.exp(k)-(1-delta)*np.exp(k)-i
	#euler equation
	eq4=np.exp(c)**(-1)-beta*np.exp(c)**(-1)*(alpha*np.exp(k)**(alpha-1)+(1-delta))
	#total factor productivity

	return [eq1,eq2,eq3,eq4]


Starting guessses.

In [5]:
yi=np.log(1)
ci=np.log(1)
ii=.1
ki=np.log(1)

initial_guess = [yi,ci,ii,ki]

This is a wrapper for the root function in scipy. It takes the initial guess and the equations function and returns the solutions. It then defines the y,c,i,k as the output and a_0 is given above as a parameter. It puts these 5 items into a dictionary.

In [6]:

def solve_equations(params):
    
    sol = root(equations, initial_guess)
    
    # Extract the solution vector
    y,c,i,k = sol.x
    #eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2 = assign_params(params)
   
    ss_dict = {
    'y':y,
    'c':c,
    'i':i,
    'k':k,
    'a_0':a_0
    }
    return sol, ss_dict
 


Calls the wrapper function. 

In [7]:
sol,ss_dict=solve_equations(ss_params)
print(sol)

 message: The solution converged.
 success: True
  status: 1
     fun: [-2.025e-11  3.178e-11  2.958e-11 -2.173e-12]
       x: [ 7.256e-01  4.795e-01  4.506e-01  2.199e+00]
    nfev: 23
    fjac: [[-6.714e-01 -6.993e-01 -1.921e-01  1.527e-01]
           [ 6.515e-01 -6.936e-01  3.003e-01  6.619e-02]
           [ 3.296e-01 -1.063e-01 -9.181e-01 -1.926e-01]
           [-1.271e-01 -1.367e-01  1.731e-01 -9.671e-01]]
       r: [-1.871e+00  5.562e-01  8.802e-01  1.442e-01  7.360e-01
            3.913e-01 -1.906e-01  9.562e-01 -2.765e-01 -8.173e-02]
     qtf: [-4.304e-10 -7.289e-10 -1.001e-09  1.367e-10]


Thus does a check. It should be 0. 

In [8]:
yss=np.exp(sol.x[0])
css=np.exp(sol.x[1])
iss=sol.x[2]
kss=np.exp(sol.x[3])

yss-css-iss


3.17785242565094e-11

This is a function to write the variables so that they are properly defined for use by dynare. See t_vars.m.

In [9]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('var \n')
        for key, value in ss_dict.items():
            f.write(f'{key}\n')
        f.write(f';\n\n\n')
        f.write('varexo \n')
        for key, value in ss_shock.items():
            f.write(f'{key}_shock\n')
        f.write(f';\n\n\n')
# Call the function
write_dict_to_file(ss_dict, 't_vars.m')

Writes the steady state values for dynare

In [10]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('initval;\n')
        for key, value in ss_dict.items():
            f.write(f'{key}={value};\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_ss.m')

Writes the parameters for dynare. 

In [11]:
def write_dict_to_file(params, filename):
    
    with open(filename, 'w') as f:
        f.write('parameters ')
        f.write(','.join(f'{key}' for key in params))
        f.write(';\n\n')
        for key, value in params.items():
            f.write(f'{key}={value};\n')

# Call the function
write_dict_to_file(params, 't_parameters.m')

Writes the shocks for Dynare

In [12]:
def write_dict_to_file(ss_dict, filename):
    #variance_list = list(shock_params.values())[:3]
    with open(filename, 'w') as f:
        f.write('shocks;\n')
        i=0
        for key, value in ss_shock.items():
            f.write(f'var {key}_shock=sigma_{key}^2;\n')
            i+=1
        f.write(f'\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_shocks.m')

Not currently used. 

In [13]:
with open('mf2.pkl', 'wb') as f:
    pickle.dump((ss_dict), f)